# Descripción

Small multiple es un tipo de visualización de datos que consiste en emplear el mismo tipo de gráfico (por ejemplo, uno de barras) para representar distintos subconjuntos de un mismo dataset, manteniendo el significado de los ejes, escalas y escalas de color entre gráficos. Estos gráficos se representan en una red o matriz de diagramas, facilitando comparar los distintos resultados. Por ejemplo, podemos representar el apoyo a las ayudas escolares en EEUU usando gráficos de mapa y representando los resultados por riqueza en el eje horizontal de la matriz y por división demográfica en el vertical (gráfico por Andrew Gelman).

![alternatvie text](https://statmodeling.stat.columbia.edu/wp-content/uploads/2009/07/vouchermapsBAYES2000.png)

Debido a que los gráficos son todos semejantes, el usuario puede rápidamente centrarse en encontrar diferencias entre los gráficos, siendo capaz de analizar los cambios en los datos y patrones. Esto los vuelven una herramienta muy útil para el análisis exploratorio de datos. Por ejemplo, podemos representar la distribución del nivel de educación para países de la UE y fácilmente encontrar aquel que tiene menos graduados universitarios en proporción a su población total.

Por otro lado, también son gráficos muy útiles a la hora de representar datos donde la granularidad de estos se pierde en una representación sencilla o en los que un gráfico quedaría demasiado aglutinado como para ser legible. Por ejemplo, si representamos la evolución del producto interior bruto de los países de la UE en un mismo gráfico, será difícil seguir todas las representaciones en un mismo diagrama, pudiendo separarlo a un driagrama por país miembro.

Sin embargo, hay que tener cuidado con el número de gráficos que creamos, pues un gran número de diagramas puede tener el efecto contrario y que el usuario se pierda entre los gráficos que se muestran. También se debe tener en cuenta cuántos gráficos representamos por cada fila y columna, pues esto afecta a la legibilidad de los gráficos.

Por otro lado hay que tener en cuenta que, al mantener la escala y ejes, los gráficos siguen teniendo los defectos que tendrían de manera individual. Por ejemplo, un small multiple diagram de gráficos de barras seguirá viéndose afectado por outliers y un exceso de categorías que mostrar. 

# Datos

https://www.ine.es/jaxiT3/Datos.htm?t=36780

Emplearemos el mismo dataset que utilizamos para los gráficos de barras.

Como recordatorio, este dataset representa los datos demográficos de España para distintos niveles de agrupación en un año determinado. En particular, se muestran datos a nivel nacional, de comunidad y de provincia divididos por sexo, nacionalidad, y edad.

En este caso, agruparemos la población por nacionalidad y sexo y mostraremos la distribución de la población de las distintas CCAA en 2022 sobre dichas agrupaciones.

También mostraremos un small multiple diagram de la distribución de población de España para dichas agrupaciones con el tiempo.

# Leer los datos

Reanizamos el mismo preprocesado: un ligero preprocesado de los datos es necesario para leer correctamente el número de individuos como número y simplificar los nombre de las variables categóricas.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def create_spanish_demographics():
    
    # Read data
    data = pd.read_csv('36780.csv',sep=';', dtype = str)
    
    # Fix data
    # CCAA names
    data['Comunidades y Ciudades Autónomas'] = data['Comunidades y Ciudades Autónomas'].str[3:]
    # Province names
    data['Provincias'] = data['Provincias'].str[3:]
    # Periodo to year
    data.Periodo = data.Periodo.str[-4:].astype(int)
    # Total to number
    data.Total   = data.Total.str.replace('.','').astype(float)
    
    return data

In [2]:
create_spanish_demographics().head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Españoles/Extranjeros,Edad (año a año),Sexo,Periodo,Total
0,Total Nacional,NaN,NaN,Total,Todas las edades,Total,2022,47475420.0
1,Total Nacional,NaN,NaN,Total,Todas las edades,Total,2021,47385107.0
2,Total Nacional,NaN,NaN,Total,Todas las edades,Total,2020,47450795.0
3,Total Nacional,NaN,NaN,Total,Todas las edades,Total,2019,47026208.0
4,Total Nacional,NaN,NaN,Total,Todas las edades,Total,2018,46722980.0


# Agrupar datos

Vamos a representar, por cada CCAA, el numero de habitantes que tiene en las posibles agrupaciones por nacionalidad (Española/Extranjero) y sexo. Esto require construir una nueva tabla de datos realizando las agrupaciones deseadas.

In [11]:
def population_ccaa_grouped(year=2022):
    
    # Load data
    data = create_spanish_demographics()
    
    # Define new category column
    data['Agrupación'] = ['Nacionalidad ' + data['Españoles/Extranjeros'][n].lower() + ' y ' + data['Sexo'][n].lower() for n in range(len(data))]
    
    # Filter data
    # By CCAA
    data = data[data['Comunidades y Ciudades Autónomas'].notna()]
    # By Province
    data = data[data['Provincias'].isna()]
    # By nationality
    data = data[data['Españoles/Extranjeros'] != 'Total']
    # By sex
    data = data[data['Sexo'] != 'Total']
    # By age
    data = data[data['Edad (año a año)'] == 'Todas las edades']
    # By year
    data = data[data['Periodo'] == year]
    
    return data

In [12]:
population_ccaa_grouped().head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Españoles/Extranjeros,Edad (año a año),Sexo,Periodo,Total,Agrupación
24500,Total Nacional,Andalucía,NaN,Española,Todas las edades,Hombres,2022,3812505.0,Nacionalidad española y hombres
24520,Total Nacional,Andalucía,NaN,Española,Todas las edades,Mujeres,2022,3946304.0,Nacionalidad española y mujeres
30620,Total Nacional,Andalucía,NaN,Extranjera,Todas las edades,Hombres,2022,375090.0,Nacionalidad extranjera y hombres
30640,Total Nacional,Andalucía,NaN,Extranjera,Todas las edades,Mujeres,2022,366288.0,Nacionalidad extranjera y mujeres
189740,Total Nacional,Aragón,NaN,Española,Todas las edades,Hombres,2022,571243.0,Nacionalidad española y hombres


# Small multiple - bar plots

Generamos ahora un small multiple diagram a base de diagramas de barras que muestren la distribución de población en función de nacionalidad y sexo para cada CCAA. Para ello creamos una función que admite indicar el número de gráficos en cada fila para ver el efecto de este parámetro en la legibilidad del diagrama.

In [41]:
import plotly
import plotly.express as px

def spanish_population_per_ccaa_small_multiple_columns(year=2022, n_cols = 5):
    
    # Load data
    data = population_ccaa_grouped(year)
    
    # Create bar plot
    fig = px.bar(data, x='Agrupación', y='Total',
                 facet_col = 'Comunidades y Ciudades Autónomas',
                 facet_col_wrap = n_cols,
                 title = 'Población por CCAA y agrupación en ' + str(year))
    
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    
    # Order figure
    fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    plotly.offline.plot(fig, 
                        filename='Población por CCAA y agrupación en ' + str(year)+ ' - ' + str(n_cols) + ' cloumnas.html')
    
    return fig

In [42]:
spanish_population_per_ccaa_small_multiple_columns()

Puede observarse claramente que Andalucía, Cataluña y Madrid son las comunidades más pobladas y que Cataluña es la comunidad en la que más extranjeros viven.

Obsérvese también que Ceuta y Melilla tienen una población muy pequeña en comparación y, debido a ello, apenas es legible su población en el gráfico. También es muy difícil apreciar los números de población extranjera en muchas comunidades debido a su baja proporción.

In [44]:
spanish_population_per_ccaa_small_multiple_columns(n_cols = 3)

Por otro lado, podemos observar que el efecto de reducir el numero de columnas se traduce en un decrimento de la legibilidad del gráfico, aglutinandose las etiquetas del eje vertical y siendo más difícil distinguir los datos de los gráficos,

# Small multiple - pie charts

También es interesante tener en cuenta que la legibilidad de un small multiple diagram puede verse muy influida por el tipo de gráfico que empleamos en su composición. Si en vez de considerar un diagrama de barras consideramos uno de tarta, es mucho más sencillo analizar la distribución de la población, aunque dejamos de ser capaces de apreciar la magnitud de cada segmento.

In [56]:
import plotly
import plotly.express as px

def spanish_population_per_ccaa_small_multiple_pies(year=2022, n_cols = 5):
    
    # Load data
    data = population_ccaa_grouped(year)
    
    # Create bar plot
    fig = px.pie(data, names='Agrupación', values='Total',
                 facet_col = 'Comunidades y Ciudades Autónomas',
                 facet_col_wrap = n_cols,
                 title = 'Población por CCAA y agrupación en ' + str(year))
    
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    
    # Save figure
    plotly.offline.plot(fig, filename='Población por CCAA y agrupación en ' + str(year)+ ' - ' + str(n_cols) + ' pie chart.html')
    
    return fig

In [57]:
spanish_population_per_ccaa_small_multiple_pies()

In [58]:
spanish_population_per_ccaa_small_multiple_pies(n_cols = 3)

# Distribución de edades por tipo de población

Finalmente, representamos la distribución de edades de las distintas agrupaciones definidas anteriormente empleando este tipo de diagramas para analizar las diferencias entre grupos de población.

In [64]:
def population_age_grouped(year=2022):
    
    # Load data
    data = create_spanish_demographics()
    
    # Define new category column
    data['Agrupación'] = ['Nacionalidad ' + data['Españoles/Extranjeros'][n].lower() + ' y ' + data['Sexo'][n].lower() for n in range(len(data))]
    
    # Filter data
    # By CCAA
    data = data[data['Comunidades y Ciudades Autónomas'].isna()]
    # By Province
    data = data[data['Provincias'].isna()]
    # By nationality
    data = data[data['Españoles/Extranjeros'] != 'Total']
    # By sex
    data = data[data['Sexo'] != 'Total']
    # By age
    data = data[data['Edad (año a año)'] != 'Todas las edades']
    data = data[data['Edad (año a año)'] != '100 y más años']
    # By year
    data = data[data['Periodo'] == year]
    
    return data

In [65]:
population_age_grouped().head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Españoles/Extranjeros,Edad (año a año),Sexo,Periodo,Total,Agrupación
6200,Total Nacional,NaN,NaN,Española,0 años,Hombres,2022,141147.0,Nacionalidad española y hombres
6220,Total Nacional,NaN,NaN,Española,0 años,Mujeres,2022,132798.0,Nacionalidad española y mujeres
6260,Total Nacional,NaN,NaN,Española,1 año,Hombres,2022,147033.0,Nacionalidad española y hombres
6280,Total Nacional,NaN,NaN,Española,1 año,Mujeres,2022,140476.0,Nacionalidad española y mujeres
6320,Total Nacional,NaN,NaN,Española,2 años,Hombres,2022,158797.0,Nacionalidad española y hombres


In [66]:
import plotly
import plotly.express as px

def spanish_population_age_dist_agrupation(year=2022):
    
    # Load data
    data = population_age_grouped(year)
    
    # Create bar plot
    fig = px.histogram(data, x='Edad (año a año)', y='Total', histnorm='probability density',
                 facet_col = 'Agrupación',
                 facet_col_wrap = 2,
                 title = 'Distribución de edades por agrupación en ' + str(year))
    
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    
    # Save figure
    plotly.offline.plot(fig, filename='Distribución de edades por agrupación en ' + str(year) + '.html')
    
    return fig

In [67]:
spanish_population_age_dist_agrupation()

Observamos que la población inmigrante es, en general, más jóven que la Española.